In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import HoverTool
from bokeh.layouts import row,column
from bokeh.io import output_notebook
#ColumnDataSource is Bokeh’s own data structure
from bokeh.models import ColumnDataSource
from bokeh.models import Div, Spinner,CustomJS, DateRangeSlider
from datetime import datetime, timedelta
from bokeh.models import DatetimeTickFormatter
from datetime import datetime
output_notebook()


Loading BokehJS ...

## Import data

These files show the number of MetroCard swipes made each week by customers entering each station of the New York City Subway, PATH, AirTrain JFK and the Roosevelt Island Tram, broken out to show the relative popularity of the various types of MetroCards. 

In [2]:
df=pd.read_csv("/Users/jchen056/Data/Fare_Card_History_for_Metropolitan_Transportation_Authority__MTA___Beginning_2010.csv")
df.head()

,From Date,To Date,Remote Station ID,Station,Full Fare,Senior Citizen / Disabled,7 Day ADA Farecard Access System Unlimited,30 Day ADA Farecard Access System Unlimited,Joint Rail Road Ticket,7 Day Unlimited,...,LIB Special Senior,Rail Road Unlimited No Trade,Transit Check Metrocard Annual Metrocard,Mail and Ride Easy Pay Express,Mail and Ride Unlimited,Path 2 Trip,Airtran Full Fare,Airtran 30 Day,Airtran 10 Trip,Airtran Monthly
0,08/06/2021,07/31/2021,R001,WHITEHALL STREET,27946,1824,242,376,8,11958,...,539,162,0,717,0,0,0,327,0,0
1,08/06/2021,07/31/2021,R003,CYPRESS HILLS,1323,134,3,13,0,817,...,12,19,0,11,0,0,0,70,0,0
2,08/06/2021,07/31/2021,R004,75TH STREET & ELDERTS LANE,4241,321,15,43,0,2029,...,14,31,0,28,0,0,0,301,0,0
3,08/06/2021,07/31/2021,R005,85TH STREET & FOREST PKWAY,4418,389,29,30,1,1999,...,13,25,0,40,0,0,0,111,0,0
4,08/06/2021,07/31/2021,R006,WOODHAVEN BOULEVARD,4193,315,27,55,0,2946,...,36,14,0,25,0,0,0,148,0,0


In [3]:
df['From_Date']=df['From Date'].apply(lambda x: datetime.strptime(x,'%m/%d/%Y'))
df['To_Date']=df['To Date'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y'))
df.head()

,From Date,To Date,Remote Station ID,Station,Full Fare,Senior Citizen / Disabled,7 Day ADA Farecard Access System Unlimited,30 Day ADA Farecard Access System Unlimited,Joint Rail Road Ticket,7 Day Unlimited,...,Transit Check Metrocard Annual Metrocard,Mail and Ride Easy Pay Express,Mail and Ride Unlimited,Path 2 Trip,Airtran Full Fare,Airtran 30 Day,Airtran 10 Trip,Airtran Monthly,From_Date,To_Date
0,08/06/2021,07/31/2021,R001,WHITEHALL STREET,27946,1824,242,376,8,11958,...,0,717,0,0,0,327,0,0,2021-08-06,2021-07-31
1,08/06/2021,07/31/2021,R003,CYPRESS HILLS,1323,134,3,13,0,817,...,0,11,0,0,0,70,0,0,2021-08-06,2021-07-31
2,08/06/2021,07/31/2021,R004,75TH STREET & ELDERTS LANE,4241,321,15,43,0,2029,...,0,28,0,0,0,301,0,0,2021-08-06,2021-07-31
3,08/06/2021,07/31/2021,R005,85TH STREET & FOREST PKWAY,4418,389,29,30,1,1999,...,0,40,0,0,0,111,0,0,2021-08-06,2021-07-31
4,08/06/2021,07/31/2021,R006,WOODHAVEN BOULEVARD,4193,315,27,55,0,2946,...,0,25,0,0,0,148,0,0,2021-08-06,2021-07-31


In [4]:
df.info()
#['14 Day Reduced Fare Media Unlimited','1 Day Unlimited','14 Day Unlimited']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250874 entries, 0 to 250873
Data columns (total 28 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   From Date                                    250874 non-null  object        
 1   To Date                                      250874 non-null  object        
 2   Remote Station ID                            250874 non-null  object        
 3   Station                                      250874 non-null  object        
 4   Full Fare                                    250874 non-null  int64         
 5   Senior Citizen / Disabled                    250874 non-null  int64         
 6   7 Day ADA Farecard Access System Unlimited   250874 non-null  int64         
 7   30 Day ADA Farecard Access System Unlimited  250874 non-null  int64         
 8   Joint Rail Road Ticket                       250874 non-null  in

In [5]:
df[df['14 Day Reduced Fare Media Unlimited'].isnull()][['From Date','To Date','14 Day Reduced Fare Media Unlimited']]

,From Date,To Date,14 Day Reduced Fare Media Unlimited
1875,11/21/2020,11/27/2020,NaN
1876,11/21/2020,11/27/2020,NaN
1877,11/21/2020,11/27/2020,NaN
1878,11/21/2020,11/27/2020,NaN
1879,11/21/2020,11/27/2020,NaN
...,...,...,...
43153,03/30/2019,04/05/2019,NaN
43154,03/30/2019,04/05/2019,NaN
43155,03/30/2019,04/05/2019,NaN
43156,03/30/2019,04/05/2019,NaN


In [6]:
df.groupby('From_Date').sum().iloc[1].sort_values()#.plot()

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_16579/2793360487.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('From_Date').sum().iloc[1].sort_values()#.plot()


Path 2 Trip                                           0.0
LIB Special Senior                                    3.0
Airtran Monthly                                     474.0
Airtran 10 Trip                                    6120.0
14 Day Reduced Fare Media Unlimited               11847.0
Airtran 30 Day                                    21670.0
Mail and Ride Unlimited                           32333.0
7 Day Express Bus Pass                            44577.0
7 Day ADA Farecard Access System Unlimited        51486.0
Joint Rail Road Ticket                            64361.0
Mail and Ride Easy Pay Express                    69209.0
Airtran Full Fare                                121878.0
Transit Check Metrocard                          143946.0
Rail Road Unlimited No Trade                     151948.0
30 Day ADA Farecard Access System Unlimited      199229.0
1 Day Unlimited                                  282755.0
14 Day Unlimited                                 578023.0
Senior Citizen

In [17]:
len(df['Remote Station ID'].unique())

525

In [4]:
# df.loc[:,"Full Fare":].corr()
full_fare_df=df.groupby('From_Date')['Full Fare'].agg(['sum','mean','std'])
days_30_Unlimited=df.groupby('From_Date')['30 Day Unlimited'].agg(['sum','mean','std'])
days_7_Unlimited=df.groupby('From_Date')['7 Day Unlimited'].agg(['sum','mean','std'])

In [36]:

# full_fare_df_new=df.groupby(['From Date','From_Date'])['Full Fare'].agg(['sum','mean','std']).reset_index()
# # full_fare_df_new.reset_index('From_Date',inplace=True)
# full_fare_df_new.set_index('From_Date',inplace=True)
# [datetime.strftime(i,"%Y-%m")for i in full_fare_df_new.index.to_list()]

['2011-01',
 '2016-01',
 '2015-01',
 '2014-01',
 '2020-01',
 '2013-01',
 '2019-01',
 '2018-01',
 '2012-01',
 '2017-01',
 '2011-01',
 '2016-01',
 '2015-01',
 '2014-01',
 '2020-01',
 '2013-01',
 '2019-01',
 '2018-01',
 '2012-01',
 '2017-01',
 '2011-01',
 '2016-01',
 '2015-01',
 '2014-01',
 '2020-01',
 '2013-01',
 '2019-01',
 '2018-01',
 '2012-01',
 '2017-01',
 '2011-01',
 '2016-01',
 '2015-01',
 '2014-01',
 '2020-01',
 '2013-01',
 '2019-01',
 '2018-01',
 '2012-01',
 '2017-01',
 '2011-01',
 '2016-01',
 '2015-01',
 '2014-02',
 '2020-02',
 '2013-02',
 '2019-02',
 '2018-02',
 '2012-02',
 '2017-02',
 '2011-02',
 '2016-02',
 '2015-02',
 '2014-02',
 '2020-02',
 '2013-02',
 '2019-02',
 '2018-02',
 '2012-02',
 '2017-02',
 '2011-02',
 '2016-02',
 '2015-02',
 '2014-02',
 '2020-02',
 '2013-02',
 '2019-02',
 '2018-02',
 '2012-02',
 '2017-02',
 '2011-02',
 '2016-02',
 '2015-02',
 '2020-02',
 '2013-02',
 '2019-02',
 '2018-02',
 '2012-02',
 '2017-02',
 '2011-02',
 '2016-02',
 '2020-02',
 '2013-03',
 '20

## Visualizations with Bohek

In [5]:
#Create a Div object and pass it some HTML code as a string:
div = Div(
    text="""
        <p>Select the circle's size using this control element:</p>
        """,
    width=200,
    height=30,
)

#Calling the figure() function
p=figure(title="Time Series",
        x_axis_label='datetime',x_axis_type="datetime",
          y_axis_label='Card Swipes',
#          y_axis_type="log",
         tools=[HoverTool()],
         tooltips=[("date","@x"),("Card Swipes","@y")],
    height=350,sizing_mode="stretch_width",)
#Use the properties of the Legend object to customize the legend
p.legend.title="Types of MetroCards"
p.legend.label_text_font="times"
p.legend.label_text_font_style = "italic"
p.legend.label_text_color = "navy"

p.title.text_font_size="25px"
p.title.align="center"
p.title.text="Time Series for Card Swipes"

p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "# of Card Swipes"
p.toolbar_location = "right"
p.toolbar.autohide = True

p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
#Adding renderers: allow you to specify visual attributes such as colors, legends, and widths
#full fare time series
p.line(full_fare_df.index,full_fare_df['sum'],
       color='blue',legend_label="Full Fare")
p.circle(full_fare_df.index,full_fare_df['sum'],
         fill_color="blue", size=2,fill_alpha=0.5,
         line_color='blue',legend_label="Full Fare")
#30 days unlimited time series
p.line(days_30_Unlimited.index, days_30_Unlimited['sum'],
      color='red',legend_label='30 Day Unlimited')
p.circle(days_30_Unlimited.index, days_30_Unlimited['sum'],
         fill_color="red",size=2,fill_alpha=0.5,
        line_color='red',legend_label='30 Day Unlimited')
#7days unlimited time series
p.line(days_7_Unlimited.index,days_7_Unlimited['sum'],legend_label="7 Day Unlimited",
      color='green',)
p.circle(days_7_Unlimited.index,days_7_Unlimited['sum'],legend_label="7 Day Unlimited",
        fill_color='green',line_color='green',size=2,fill_alpha=0.5)
#Asking Bokeh to show() or save() the results
show(p)

/Users/jchen056/opt/anaconda3/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.title` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/Users/jchen056/opt/anaconda3/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.label_text_font` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/Users/jchen056/opt/anaconda3/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.label_text_font_style` on a plot that has zero legends added, this will have no effect.

B

In [6]:
# set output to static HTML file
output_file(filename="Visualizations/payment_timeseries.html")

#Calling the figure() function
p=figure(title="Time Series",
        x_axis_label='datetime',x_axis_type="datetime",
          y_axis_label='Card Swipes',
#          y_axis_type="log",
         tools=[HoverTool()],
         tooltips=[("date","@x_values"),("Card Swipes","@y_values)")],
    height=350,sizing_mode="stretch_width",)
#Use the properties of the Legend object to customize the legend
p.legend.title="Types of MetroCards"
p.legend.label_text_font="times"
p.legend.label_text_font_style = "italic"
p.legend.label_text_color = "navy"

p.title.text_font_size="25px"
p.title.align="center"
p.title.text="Time Series for Card Swipes"

p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "# of Card Swipes"
p.toolbar_location = "right"
p.toolbar.autohide = True

p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")

payment_types=["Full Fare",'30 Day Unlimited','7 Day Unlimited']
colors=['blue','red','green']
for i,payment_type in enumerate(payment_types):
    df_payment=df.groupby('From_Date')[payment_type].agg(['sum','mean','std'])
    #ColumnDataSource is Bokeh’s own data structure
    data={"x_values":df_payment.index,"y_values":df_payment['sum'],"datestr":
          [datetime.strftime(i,"%Y-%m-%d")for i in df_payment.index.to_list()]}
    source = ColumnDataSource(data=data)# create ColumnDataSource based on dict
    p.line(x='x_values', y='y_values',source=source,
          color=colors[i],legend_label=payment_type)
    p.circle(x='x_values', y='y_values',source=source,
             fill_color=colors[i],size=2,fill_alpha=0.5,
        line_color=colors[i],legend_label=payment_type
            )
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("date","@datestr"),("Card Swipes","@y_values)")]
hover.mode = 'mouse'

# date_range_slider = DateRangeSlider(value=(df['From_Date'].min(),df['From_Date'].max()),
#                                     start=df['From_Date'].min(), 
#                                     end=df['From_Date'].max())
# date_range_slider.js_on_change("value", CustomJS(code="""
#     console.log('date_range_slider: value=' + this.value, this.toString())
# """))

save(p)

'/Users/jchen056/MTA_MTA/Visualizations/payment_timeseries.html'

In [21]:
#https://new.mta.info/agency/new-york-city-transit/subway-bus-ridership-2022
# output_file(filename="Visualizations/annual_ridership_time_series.html")
x = [2017,2018,2019,2020,2021,2022]
y = [1727366607,1680060402, 1697787002,
    639541029, 759976721,1013425465
]

data = {'Year':x,
        'Annual_Subway_Ridership': y}
source = ColumnDataSource(data=data)

# create a new plot with a title and axis labels
p1= figure(title="Annual Subway Ridership", 
           x_axis_label='Year', y_axis_label='Annual Subway Ridership',
           tools=[HoverTool()],
         tooltips=[("Year","@Year"),("Annual Subway Ridership","@Annual_Subway_Ridership")],
          height=350,sizing_mode="stretch_width")
p1.vbar(x='Year', top='Annual_Subway_Ridership',source=source,
        width=0.5, bottom=0, color="blue")
p1.line(x='Year', y='Annual_Subway_Ridership',source=source,
        line_width=2,color='red')
p1.circle(x='Year', y='Annual_Subway_Ridership',source=source,
          line_width=4,color='red',
        size=2,fill_alpha=0.5,
        line_color='red')

hover1 = p.select(dict(type=HoverTool))
hover1.tooltips = [("Year","@Year"),("Annual Subway Ridership","@Annual_Subway_Ridership")]
hover1.mode = 'mouse'

'/Users/jchen056/MTA_MTA/Visualizations/payment_timeseries.html'

In [24]:
show(p1)
save(p1)

'/Users/jchen056/MTA_MTA/Visualizations/annual_ridership_time_series.html'